In [25]:
import asyncio
import os
from datetime import datetime, timedelta
from typing import List, Any

import aiohttp
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
from sklearn.linear_model import LinearRegression
import numpy as np

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBRegressor

In [2]:
def clean_crypto_data(csv_file_path):
    """
    Cleans the cryptocurrency data in the specified CSV file by removing any columns
    associated with a coin if any of its values are empty or None.

    Parameters:
    csv_file_path (str): The path to the CSV file.

    Returns:
    None: The function updates the CSV file in place.
    """
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path, index_col=0)

    # Create a set to keep track of columns to drop
    columns_to_drop = set()

    # Get unique coin names from the column names
    coin_names = set(col.split('_')[1] for col in df.columns if '_' in col)

    # Iterate through each coin and check its associated columns
    for coin in coin_names:
        # Get all columns associated with this coin
        associated_columns = [col for col in df.columns if f'_{coin}' in col]
        
        # Check if any of these columns contain null or empty values
        if df[associated_columns].isnull().any().any() or (df[associated_columns] == '').any().any():
            columns_to_drop.update(associated_columns)

    # Drop identified columns
    df.drop(columns=columns_to_drop, inplace=True)

    # Save the cleaned DataFrame back to CSV
    df.to_csv(csv_file_path, index=False)

In [3]:
def calculate_returns(df, return_timeframes):
    """
    Adds return features to the DataFrame for each coin based on specified return time frames.

    Parameters:
    df (pd.DataFrame): The DataFrame containing cryptocurrency data with timestamp and open values.
    return_timeframes (list): A list of time frames in hours for which to calculate returns.

    Returns:
    pd.DataFrame: The updated DataFrame with return features added.
    """
    # Ensure the DataFrame is sorted by timestamp
    # df.sort_values(by='timestamp', inplace=True)

    # Create a copy of the DataFrame to avoid modifying the original
    updated_df = df.copy()

    # Iterate through each coin's columns
    coin_columns = [col for col in df.columns if '_' in col]
    incr = 0
    for coin in set(col.split('_')[1] for col in coin_columns):
        open_col = f'open_{coin}'
        # print(coin, incr)
        incr += 1
        for hours in return_timeframes:
            # Calculate the number of rows to go back based on the frequency (1 row per minute)
            rows_to_go_back = hours * 60
            
            # Create a new column for returns
            return_col_name = f'return{hours}_{coin}'
            updated_df[return_col_name] = pd.NA
            
            # Calculate return values using vectorized operations
            updated_df[return_col_name] = updated_df[open_col] - updated_df[open_col].shift(rows_to_go_back)
    
    updated_df =  updated_df[rows_to_go_back + 1:]

    return updated_df

In [4]:
def remove_rows_with_nan(df):
    """
    Removes any rows from the DataFrame that contain an 'x' in any column.

    Parameters:
    df (pd.DataFrame): The DataFrame from which to remove rows.

    Returns:
    pd.DataFrame: The cleaned DataFrame without rows containing 'x'.
    """
    
    # Use boolean indexing to filter out rows containing 'x'
    cleaned_df = df[df.notna().all(axis=1)]
    cleaned_df = cleaned_df[~df.isin(['x']).any(axis=1)]

    return df

In [5]:
def get_top_correlated_cryptos(df, crypto_name, top_n=20):
    """
    This function takes a DataFrame of cryptocurrency returns and a specific cryptocurrency name,
    and returns a list of the top N cryptocurrencies most correlated with it.

    Parameters:
    df (pd.DataFrame): DataFrame containing cryptocurrency returns.
    crypto_name (str): The name of the cryptocurrency to compare against.
    top_n (int): The number of top correlated cryptocurrencies to return.

    Returns:
    List[str]: A list of top N correlated cryptocurrency names.
    """
    
    # Find columns that contain the specified crypto_name
    matching_columns = df.columns[df.columns.str.contains(crypto_name, case=False)]
    
    # Check if any matching columns were found
    if len(matching_columns) == 0:
        raise ValueError(f"No columns found containing '{crypto_name}'.")
    
    # Use the first matching column for correlation analysis
    target_column = matching_columns[0]
    
    # Calculate correlation matrix
    correlation_matrix = df.corr()
    
    # Get correlations for the specified cryptocurrency
    crypto_correlations = correlation_matrix[target_column]
    
    # Sort correlations in descending order and exclude self-correlation
    sorted_correlations = crypto_correlations.drop(target_column).sort_values(ascending=False)
    
    # Get top N correlated cryptocurrencies
    top_correlated_cryptos = sorted_correlations.head(top_n).index.tolist()
    
    return top_correlated_cryptos
    

In [6]:
csv_file_path = "crypto_data.csv"
return_features = [1,12,24]
df = pd.read_csv(csv_file_path)
coin_name = "BNBUSDT"
# Calculate returns for different time frames
return_df = calculate_returns(df, return_features)
# print(return_df["return24_BNBUSDT"])
# Remove rows containing 'x'
cleaned_df = remove_rows_with_nan(return_df)
# print(cleaned_df)
returnsDF = cleaned_df.filter(regex='return24')
# print(returnsDF["return24_BNBUSDT"])
top_correlated = get_top_correlated_cryptos(returnsDF, "BNBUSDT")
print(top_correlated)

/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2822795501.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  updated_df[return_col_name] = pd.NA
/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2822795501.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  updated_df[return_col_name] = pd.NA
/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2822795501.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

['return24_BNBPERP', 'return24_CAKEUSDT', 'return24_XVSUSDT', 'return24_NKNUSDT', 'return24_BTCUSDT', 'return24_BTC-28MAR25', 'return24_BTC-27JUN25', 'return24_BTC-27DEC24', 'return24_BTCPERP', 'return24_BTC-25OCT24', 'return24_ARPAUSDT', 'return24_IOTXUSDT', 'return24_ANKRUSDT', 'return24_UNFIUSDT', 'return24_ASTRUSDT', 'return24_FLOWUSDT', 'return24_GASUSDT', 'return24_HIFIUSDT', 'return24_AXSUSDT', 'return24_BATUSDT']


In [7]:
get_name = lambda x: x.split("_")[1]
coins = [get_name(x) for x in top_correlated]
correlated_df = returnsDF[top_correlated]
correlated_df[f'open_{coin_name}'] = cleaned_df[f'open_{coin_name}']
correlated_df[f'return24_BTCUSDT'] = returnsDF[f'return24_BTCUSDT']
correlated_df[f'return24_BNBUSDT'] = returnsDF[f'return24_BNBUSDT']
correlated_df['time'] = cleaned_df['time']
correlated_df.set_index('time', inplace=True)
print(correlated_df)


                     return24_BNBPERP  return24_CAKEUSDT  return24_XVSUSDT  \
time                                                                         
2024-09-02 00:01:00            -12.95             -0.041            -0.226   
2024-09-02 00:02:00            -12.20             -0.043            -0.230   
2024-09-02 00:03:00            -11.50             -0.044            -0.216   
2024-09-02 00:04:00            -11.50             -0.042            -0.214   
2024-09-02 00:05:00            -11.50             -0.042            -0.216   
...                               ...                ...               ...   
2024-09-13 23:56:00             13.65              0.035             0.039   
2024-09-13 23:57:00             13.40              0.032             0.039   
2024-09-13 23:58:00             13.65              0.035             0.043   
2024-09-13 23:59:00             13.85              0.034             0.045   
2024-09-14 00:00:00             13.40              0.033        

/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2970399933.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correlated_df[f'open_{coin_name}'] = cleaned_df[f'open_{coin_name}']
/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2970399933.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correlated_df[f'return24_BTCUSDT'] = returnsDF[f'return24_BTCUSDT']
/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2970399933.py:6: SettingWithCopyWarning: 
A value is t

In [8]:
def find_betas(old_df, train_size):
    # Read the CSV file
    df = old_df.copy()
    # df.set_index('time', inplace=True)

    # Get unique coins
    coins = [col.split('_')[1] for col in df.columns if col.startswith('return24_')]
    coins.remove('BTCUSDT')

    # Calculate beta values
    beta_values = {}
    for coin in coins:
        X = df['return24_BTCUSDT'].values[:train_size].reshape(-1, 1)
        y = df[f'return24_{coin}'].values[:train_size]
        model = LinearRegression().fit(X, y)
        beta_values[coin] = model.coef_[0]
    
    return beta_values

def normalize_to_btcusdt(df, beta_values):
    # Normalize data
    coins = [col.split('_')[1] for col in df.columns if col.startswith('return24_')]
    coins.remove('BTCUSDT')
    for coin in coins:
        col_name = f'return24_{coin}'
        # print(beta_values[coin])
        df[col_name] = df[col_name] - beta_values[coin] * df[f'return24_BTCUSDT']

    return df

In [9]:
def add_future_return_target(df, return_timeframe, crypto):
    """
    Adds a target column to the DataFrame for the return value x hours into the future for a specified cryptocurrency.

    Parameters:
    df (pd.DataFrame): The DataFrame containing cryptocurrency data with timestamp and open values.
    return_timeframe (int): The time frame in hours for which to calculate future returns.
    crypto (str): The cryptocurrency for which to calculate the future return.

    Returns:
    pd.DataFrame: The updated DataFrame with the target column added.
    """
    
    # Ensure the DataFrame is sorted by timestamp
    # df.sort_index(inplace=True)

    # Create a copy of the DataFrame to avoid modifying the original
    updated_df = df.copy()

    # Determine the open value column for the specified cryptocurrency
    return_col = f'return24_{crypto}'
    
    # Calculate the number of rows to go forward based on the frequency (1 row per minute)
    rows_to_go_forward = return_timeframe * 60

    # Create a new column for future returns
    target_col_name = f'target_{crypto}'
    updated_df[target_col_name] = updated_df[return_col].shift(-rows_to_go_forward)

    # Fill NaN values at the beginning with 'x'
    n = rows_to_go_forward
    updated_df = updated_df[:-n]
    # updated_df.loc[df.index[-n:], target_col_name] = 'x'
    # updated_df.iloc[:rows_to_go_forward, target_col_name] = pd.NA

    return updated_df

In [10]:
train_size = int(0.8 * (len(correlated_df) - 24*60))
betas = find_betas(correlated_df, train_size)
print(betas)
normalized_df = normalize_to_btcusdt(correlated_df, betas)
print("done")
normalized_df = remove_rows_with_nan(add_future_return_target(normalized_df, 24, coin_name))
# normalized_df = normalized_df.apply(pd.to_numeric, errors='coerce').dropna()
# non_numeric_values = df[df['open_BNBUSDT'].apply(lambda x: not pd.to_numeric(x, errors='coerce').notnull())]

# Print the non-numeric values
# print(non_numeric_values)
train_df = normalized_df[:train_size]
test_df = normalized_df[train_size:]

print(train_df)
print(test_df)

{'BNBPERP': 0.007882485730681761, 'CAKEUSDT': 3.101063922440301e-05, 'XVSUSDT': 0.0001253699809710267, 'NKNUSDT': 1.45969653552894e-06, 'BTC-28MAR25': 1.078444568808483, 'BTC-27JUN25': 1.1102420889053521, 'BTC-27DEC24': 1.0438064587963123, 'BTCPERP': 1.0062721263262577, 'BTC-25OCT24': 1.0207988533586643, 'ARPAUSDT': 8.141205657579908e-07, 'IOTXUSDT': 7.453270648951808e-07, 'ANKRUSDT': 4.871640661709182e-07, 'UNFIUSDT': 5.25228939052455e-05, 'ASTRUSDT': 1.3321554308750647e-06, 'FLOWUSDT': 1.147375249895237e-05, 'GASUSDT': 5.134601185838918e-05, 'HIFIUSDT': 8.981722489067975e-06, 'AXSUSDT': 8.847180094964497e-05, 'BATUSDT': 2.694471165625357e-06, 'BNBUSDT': 0.007836990871545603}
done


/var/folders/3b/0jn7ny8n0792y6v1yr47g2m40000gn/T/ipykernel_22436/2608901826.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df[col_name] - beta_values[coin] * df[f'return24_BTCUSDT']


                     return24_BNBPERP  return24_CAKEUSDT  return24_XVSUSDT  \
time                                                                         
2024-09-02 00:01:00         -6.890733          -0.017162         -0.129628   
2024-09-02 00:02:00         -5.239765          -0.015618         -0.119298   
2024-09-02 00:03:00         -4.663520          -0.017104         -0.107267   
2024-09-02 00:04:00         -4.643026          -0.015024         -0.104941   
2024-09-02 00:05:00         -5.179035          -0.017133         -0.115466   
...                               ...                ...               ...   
2024-09-10 19:08:00         -2.119512           0.006777          0.023456   
2024-09-10 19:09:00         -2.279527           0.005147          0.015911   
2024-09-10 19:10:00         -2.173113           0.005566          0.017603   
2024-09-10 19:11:00         -1.800272           0.007033          0.021533   
2024-09-10 19:12:00         -2.022558           0.006158        

In [30]:
def predict_crypto_returns(train_df, test_df, target_coin, target_hours, train_size = 0.8, random_state=42):
    """
    Create a LightGBM model to predict target return values for a specific coin and time horizon.
    
    :param df: DataFrame containing cryptocurrency data
    :param target_coin: The coin to predict returns for (e.g., 'BTC')
    :param target_hours: The time horizon for returns (e.g., 2, 6, or 12)
    :param test_size: Proportion of data to use for testing (default: 0.2)
    :param random_state: Random state for reproducibility (default: 42)
    :return: Trained model, test predictions, and evaluation metrics
    """
    
    # Identify the target column
    target_col = f'target_{target_coin}'
    # df.drop(columns = ["time"], inplace = True)
    if target_col not in train_df.columns:
        raise ValueError(f"Target column {target_col} not found in the dataframe.")
    
    # Separate features and target
    feature_cols = [col for col in train_df.columns if col != target_col and not col.startswith('target')]
    
    # X = dim_reduction(X)
    # Split the data
    X_train = train_df[feature_cols]
    y_train = train_df[target_col]
    X_test = test_df[feature_cols]
    y_test = test_df[target_col]
    params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'device': 'gpu'  # Use GPU for training
    }
    train_data = lgb.Dataset(X_train, label=y_train)
    # Initialize and train the model
    model = XGBRegressor(objective='reg:squarederror', n_estimators=50)
    model.fit(X_train, y_train)
    # model = lgb.train(params, train_data, num_boost_round=100)
    # model = LGBMRegressor(random_state=random_state)
    # model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_2 = [0] * len(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Evaluate the model
    mse_2 = mean_squared_error(y_test, y_pred_2)
    mae_2 = mean_absolute_error(y_test, y_pred_2)
    rmse_2 = np.sqrt(mse)
    r2_2 = r2_score(y_test, y_pred_2)

    print(mse_2, mae_2, rmse_2, r2_2, "fake")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    print(feature_importance)
    
    return {
        'model': model,
        'predictions': y_pred,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mae': mae
    }

In [31]:
model = predict_crypto_returns(train_df, test_df, coin_name, 24)
print(model)

125.78082476516515 8.189790347266186 11.105607032870475 -0.5382022711888199 fake
                 feature  importance
21      return24_BNBUSDT    0.160464
20          open_BNBUSDT    0.159761
16      return24_GASUSDT    0.121538
18      return24_AXSUSDT    0.114393
2       return24_XVSUSDT    0.082735
1      return24_CAKEUSDT    0.069469
4       return24_BTCUSDT    0.056633
19      return24_BATUSDT    0.045670
14     return24_ASTRUSDT    0.044724
10     return24_ARPAUSDT    0.032639
0       return24_BNBPERP    0.027827
3       return24_NKNUSDT    0.019866
12     return24_ANKRUSDT    0.019156
11     return24_IOTXUSDT    0.017571
15     return24_FLOWUSDT    0.011458
17     return24_HIFIUSDT    0.008262
13     return24_UNFIUSDT    0.005292
5   return24_BTC-28MAR25    0.001344
7   return24_BTC-27DEC24    0.000331
9   return24_BTC-25OCT24    0.000313
6   return24_BTC-27JUN25    0.000309
8       return24_BTCPERP    0.000245
{'model': XGBRegressor(base_score=None, booster=None, callbacks=None